# 년 데이터 -> 월/분기/반기 변환

---------------
## 0. 환경설정

In [1]:
##### 라이브러리 호출 #####
import numpy as np
import pandas as pd
import time
import glob
import pickle
import itertools
import copy

import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# 데이터프레임 출력 옵션
pd.set_option('display.max_columns', 100)

#지수표현
pd.options.display.float_format = '{:.5f}'.format

----------
## 1. 입력값 기입

In [2]:
## data_folder : 원본데이터 위치(폴더명)
## save_folder : 변환데이터 저장 위치(폴더명)
## file_nm : 파일명
## transform_range : 데이터 저장범위 / 'all' / 'month' / 'quarter' / 'half' 중에 선택하여 입력('all' 입력 시 월/분기/반기 결과 모두 저장)
## target : 타겟변수(값 변환시키지 않을 str 타입 컬럼)
## year_colnm : 연도 컬럼명
## data_file_encoding : 데이터 파일의 인코딩 명

# data_folder = 'data'
# save_folder = 'data/transform'
# file_nm = ['merge_age_new','merge_cls_new','merge_dis_new','merge_part_age','merge_part_dis9']
# transform_range = ['all','all','all','all','all']
# target = [['SIDO','SCHOOL_TP'],['SIDO'],['SIDO','DIS_TYPE'],['SIDO_EDU_NM','PART_EDU_NM','PART_EDU_CD','SCHOOL_TP'],['PART_EDU_NM','PART_EDU_CD']]
# year_colnm = ['BASE_YY','BASE_YY','BASE_YY','BASE_YY','BASE_YY']
# data_file_encoding = ['cp949','cp949','cp949','cp949','cp949']

In [3]:
# (임시) 예시 - 삭제 가능
data_folder = 'data'
save_folder = 'data/transform'
file_nm = ['merge_cls_new']
transform_range = ['all']
target = [['SIDO']]
year_colnm = ['BASE_YY']
data_file_encoding = ['cp949']

----------------
## 2. 데이터 변환

### 2-1. 함수 선언

In [4]:
# 연도 컬럼을 이용하여 (연도/월) 컬럼 생성
def fn_yr_mon_col(data):
    
    # (1) 연도 컬럼 생성
    year_list = list(set(data['YEAR']))
    year_list.sort()

    for year_nm in year_list:
        if year_nm == year_list[0]:
            nujuk_year = [str(year_nm)]
        else:
            year_nm = str(year_nm) + ' '
            year_nm = ((year_nm * 12).split(' '))[:-1]
            nujuk_year = nujuk_year + year_nm
            
    nujuk_year = pd.DataFrame(nujuk_year, columns = ['YEAR'])
    nujuk_year['YEAR'] = nujuk_year['YEAR'].astype('str')

    # (2) 월 컬럼 생성
    nujuk_month = ['12'] + (len(year_list) - 1) * list(map(str, range(1,13)))
    nujuk_month = pd.DataFrame(nujuk_month, columns = ['MONTH'])
    nujuk_month['MONTH'] = nujuk_month['MONTH'].astype('str')
    nujuk_month['MONTH'] = list(map((lambda x : '0' + str(x) if int(x) < 10 else str(x)), nujuk_month['MONTH']))
            
    # (3) (연도/월) 컬럼 병합
    nujuk_year_month = pd.concat([nujuk_year, nujuk_month], axis = 1)
    
    return nujuk_year_month

In [5]:
# 연도 값을 이용하여 월 값 생성
def fn_year_to_month(data, float_col):

    for chg_colnm in float_col:

        # 데이터 정의
        col_data = data[['YEAR', chg_colnm]]
        col_data['diff_12'] = ((col_data[chg_colnm] - col_data[chg_colnm].shift(1)) / 12)

        ## [1] 연도 값을 월 값으로 확장
        yr_val_list = list(col_data[chg_colnm])

        for yr_num in range(len(yr_val_list)):
            if yr_num == 0:       # 첫번째 연값
                yr_val = str(yr_val_list[yr_num]) + ' '
                year_value = (yr_val * 13).split(' ')[:-1]
            elif yr_num == (len(yr_val_list) - 1):    # 마지막 연값
                pass
            else:
                yr_val = str(yr_val_list[yr_num]) + ' '     # 첫번째 또는 마지막 연값이 아닌 경우
                year_value = year_value + (yr_val * 12).split(' ')[:-1]
                
        year_value = pd.DataFrame(year_value, columns = [chg_colnm])
        year_value[chg_colnm] = year_value[chg_colnm].astype('float')

        ## [2] 월별 add 값 생성
        # (2-1) diff/12
        add_val_list = list(col_data['diff_12'][1:])
                
        for add_num in range(len(add_val_list)):
            if add_num == 0:
                add_val = str(add_val_list[add_num]) + ' '
                add_value = ['0'] + (add_val * 12).split(' ')[:-1]
            else:
                add_val = str(add_val_list[add_num]) + ' '
                add_value = add_value + (add_val * 12).split(' ')[:-1]
                
        add_value = pd.DataFrame(add_value, columns = ['add_value'])
        add_value['add_value'] = add_value['add_value'].astype('float')

        # (2-2) 1 ~ 12
        multi_value = pd.DataFrame([0] + (len(data) -1) * list(range(1,13)), columns = ['multi_value'])
        multi_value['multi_value'] = multi_value['multi_value'].astype('float')

        # (2-3) add 값 생성 : (2-1) * (2-2)
        mon_add_value = pd.DataFrame(add_value['add_value'] * multi_value['multi_value'], columns = ['mon_add_value'])
        
        ## [3] 월 값 생성 : [1] + [2]
        fin_year_value = pd.DataFrame(year_value[chg_colnm] + mon_add_value['mon_add_value'], columns = [chg_colnm])
        
        # (연도/월) 컬럼 호출 : 함수 이용
        nujuk_year_month = fn_yr_mon_col(data)
        
        # (연도/월/생성 월 값) 병합
        if chg_colnm == float_col[0]:
            nujuk_fin_year_value = pd.concat([nujuk_year_month, fin_year_value], axis = 1)
        else:
            nujuk_fin_year_value = pd.concat([nujuk_fin_year_value, fin_year_value], axis = 1)
        
    return nujuk_fin_year_value

### 2-2. 변환 수행

In [6]:
# 파일 정보 통합
file_info = pd.DataFrame(file_nm).rename(columns = {0:'file_nm'})
file_info['transform_range'] = transform_range
file_info['target'] = target
file_info['year_colnm'] = year_colnm
file_info['data_file_encoding'] = data_file_encoding

for file_num in range(len(file_info)):

    # 분석 정보 호출
    file_nm = file_info['file_nm'][file_num]
    transform_range = file_info['transform_range'][file_num]
    target = file_info['target'][file_num]
    year_colnm = file_info['year_colnm'][file_num]
    data_file_encoding = file_info['data_file_encoding'][file_num]

    # 데이터 호출
    tot_data = pd.read_csv(data_folder + '/' + file_nm + '.csv', encoding = data_file_encoding, dtype = 'str')
    tot_data.rename(columns = {year_colnm:'YEAR'}, inplace=True)  # 연도 컬럼명 변경        

    # target 리스트를 병합하여 하나의 컬럼('target')으로 생성 (ex. 강원,특수교육 => 강원_특수교육)
    for trg_num in range(len(target)):
        if len(target) == 1:  # str 컬럼이 하나인 경우
            tot_data['target'] = tot_data[target[trg_num]]
        else:
            if trg_num == 0:  # str 컬럼이 여러개인 경우
                tot_data['target'] = tot_data[target[trg_num]]
            else:
                tot_data['target'] = tot_data['target'] + '_' + tot_data[target[trg_num]]

    # 데이터 형 변환(str -> float) : target 컬럼을 제외한 나머지 컬럼들은 월 값 변환을 위해 float 형식으로 변경
    str_col = ['YEAR','target'] + target
    float_col = list(set(tot_data.columns).difference(set(str_col)))

    tot_data[float_col] = tot_data[float_col].astype('float')

    # 생성 월값 누적할 데이터프레임
    nujuk_yr_to_mon_data = pd.DataFrame()

    # loop_target : 타겟 리스트
    loop_target = list(set(tot_data['target']))
    loop_target.sort()
    
    for trg in loop_target:

        # 타겟의 연값 호출
        data = tot_data.loc[tot_data['target'] == trg,]
        data = data.sort_values(by = 'YEAR').reset_index(drop=True)

        # 타겟의 연값을 월값으로 변환 : 함수 이용
        nujuk_fin_year_value = fn_year_to_month(data, float_col)
        
        # 컬럼 순서 변경
        col_list = ['target'] + list(nujuk_fin_year_value.columns)            
        nujuk_fin_year_value['target'] = trg
        nujuk_fin_year_value = nujuk_fin_year_value[col_list]

        # (타겟별) 생성 월값 누적
        nujuk_yr_to_mon_data = nujuk_yr_to_mon_data.append(nujuk_fin_year_value)

    # --------------------------------------------------------------------------------------------------------------- #
    ## 월 데이터
    month_data = copy.deepcopy(nujuk_yr_to_mon_data)
    # --------------------------------------------------------------------------------------------------------------- #
    ## 분기 데이터
    quarter_data = copy.deepcopy(nujuk_yr_to_mon_data)
    
    quarter_data.loc[quarter_data['MONTH'].isin(['01','02','04','05','07','08','10','11']),'QUARTER'] = '00'        
    quarter_data.loc[quarter_data['MONTH'].isin(['03']),'QUARTER'] = '01'
    quarter_data.loc[quarter_data['MONTH'].isin(['06']),'QUARTER'] = '02'
    quarter_data.loc[quarter_data['MONTH'].isin(['09']),'QUARTER'] = '03'
    quarter_data.loc[quarter_data['MONTH'].isin(['12']),'QUARTER'] = '04'

    # 분기 값이 아닌 경우 제외
    quarter_data = quarter_data.loc[quarter_data['QUARTER'] != '00',]
    
    # 컬럼 순서 변경
    col_list = set(list(quarter_data.columns)).difference(('target','YEAR','MONTH','QUARTER'))
    col_list = ['target','YEAR','QUARTER'] + list(col_list)
    quarter_data = quarter_data[col_list]
    # --------------------------------------------------------------------------------------------------------------- #
    ## 반기 데이터
    half_data = copy.deepcopy(nujuk_yr_to_mon_data)

    half_data.loc[half_data['MONTH'].isin(['01','02','03','04','05','07','08','09','10','11']),'HALF'] = '00'
    half_data.loc[half_data['MONTH'].isin(['06']),'HALF'] = '01'
    half_data.loc[half_data['MONTH'].isin(['12']),'HALF'] = '02'
    
    # 반기 값이 아닌 경우 제외
    half_data = half_data.loc[half_data['HALF'] != '00',]  

    # 컬럼 순서 변경
    col_list = set(list(half_data.columns)).difference(('target','YEAR','MONTH','HALF'))
    col_list = ['target','YEAR','HALF'] + list(col_list)
    half_data = half_data[col_list] 
    # --------------------------------------------------------------------------------------------------------------- #
    ## 연 데이터
    year_data = tot_data[['target','YEAR'] + list(set(tot_data.columns).difference(set(['target','YEAR'] + target)))]
    # --------------------------------------------------------------------------------------------------------------- #
    ## 결과 저장
    if transform_range == 'all':
        save_list = ['month','quarter','half','year']
        for s_lst in save_list:
            globals()[s_lst + '_data'].to_csv(save_folder + '/' + s_lst + '_' + file_nm + '.csv', index = False, encoding = 'utf-8')
    elif transform_range == 'month':
        save_list = ['month','year']
        for s_lst in save_list:
            globals()[s_lst + '_data'].to_csv(save_folder + '/' + s_lst + '_' + file_nm + '.csv', index = False, encoding = 'utf-8')
    elif transform_range == 'quarter':
        save_list = ['quarter','year']
        for s_lst in save_list:
            globals()[s_lst + '_data'].to_csv(save_folder + '/' + s_lst + '_' + file_nm + '.csv', index = False, encoding = 'utf-8')
    elif transform_range == 'half':
        save_list = ['half','year']
        for s_lst in save_list:
            globals()[s_lst + '_data'].to_csv(save_folder + '/' + s_lst + '_' + file_nm + '.csv', index = False, encoding = 'utf-8')
    else:
        print('월/분기/반기 모두 저장합니다.')
        save_list = ['month','quarter','half','year']
        for s_lst in save_list:
            globals()[s_lst + '_data'].to_csv(save_folder + '/' + s_lst + '_' + file_nm + '.csv', index = False, encoding = 'utf-8')

    print('(월/분기/반기) 변환 종료')

(월/분기/반기) 변환 종료


--------